In [1]:
from EBSD2GRID import EBSD2GRID
import numpy as np
import damask
import pandas as pd
import yaml

In [2]:
ctf_file = "A55-100.ctf"
EBSD2GRID(ctf_file, 2) # 第二个参数是相数

In [3]:
Fix_data = pd.read_csv("A55-100.csv", sep='\t', dtype=str)

# phase = 0 的 都改成 1
# 相当于消除领解析的相

Fix_data.loc[Fix_data['phase'] == '0', 'phase'] = '1'


# phase = 1 的都改为 相名称
Fix_data.loc[Fix_data['phase'] == '1', 'phase'] = 'IronBcc'
Fix_data.loc[Fix_data['phase'] == '2', 'phase'] = 'IronFcc' #如果有第二相

Fix_data.to_csv("A55-100.csv", index=False, sep=' ')

In [4]:
with open("Hooke_Fe.yaml", 'r', encoding='utf-8') as f:
    Fe_elasticity = yaml.safe_load(f)

with open("phenopowerlaw_DP-steel-ferrite.yaml", 'r', encoding='utf-8') as f:
    IronBCC_plasticity = yaml.safe_load(f)

# 这是镍基合金的 参数得改
with open("phenopowerlaw_IN625.yaml", 'r', encoding='utf-8') as f:
    IronFcc_plasticity = yaml.safe_load(f)

homogenization = {
    'SX': {
        'N_constituents': 1,
        'mechanical': {'type': 'pass'}  # No volume fractions, so no homogenization needed
    }
}

# 会多一个Cu, 删了就行
# phase_Cu = {
#     'Cu': {
#         'lattice': 'cF',
#         'rho' : 8960.0,
#         'mechanical': {
#             'output': ['F', 'P', 'F_e', 'L_p', 'O'],
#             'elastic': Cu_elasticity,
#             'plastic': Cu_plasticity
#         }
#     }
# }

phase_IronBcc = {
        'lattice': 'cI',
        'rho' : 7874.0,
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': Fe_elasticity,
            'plastic': IronBCC_plasticity
    }
}

phase_IronFcc = {
        'lattice': 'cF',
        'rho' : 7874.0,
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': Fe_elasticity,
            'plastic': IronFcc_plasticity
    }
}

In [13]:
Model = damask.Table.load("A55-100.csv")

gridfile = damask.GeomGrid.from_table(Model, 'pos', ['phase', 'qu'])
gridfile.origin = [0, 0, 0]
gridfile.size[2] = 0.05
CM = damask.ConfigMaterial.from_table(Model, phase='phase', O='qu', homogenization='homogenization')
CM['phase']['IronFcc'] = phase_IronFcc
CM['phase']['IronBcc'] = phase_IronBcc
CM['homogenization']= homogenization

In [14]:
CM['phase']

{'IronBcc': {'lattice': 'cI',
  'rho': 7874.0,
  'mechanical': {'output': ['F', 'P', 'F_e', 'L_p', 'O'],
   'elastic': {'type': 'Hooke',
    'references': ['D.J. Dever, Journal of Applied Physics 43(8):3293-3301, 1972, https://doi.org/10.1063/1.1661710, fit to Tab. II (T_min=25ºC, T_max=880ºC)'],
    'C_11': 232200000000.0,
    'C_11,T': -46830000.0,
    'C_11,T^2': -59880.0,
    'C_12': 136400000000.0,
    'C_12,T': -19700000.0,
    'C_12,T^2': 3760.0,
    'C_44': 117000000000.0,
    'C_44,T': -20150000.0,
    'C_44,T^2': -748.5,
    'T_ref': 293.15},
   'plastic': {'type': 'phenopowerlaw',
    'references': ['C.C. Tasan et al., Acta Materialia 81:386-400, 2014, https://doi.org/10.1016/j.actamat.2014.07.071, Table 1',
     'U.F. Kocks, Metallurgical and Materials Transactions B 1:1121–1143, 1970, https://doi.org/10.1007/BF02900224'],
    'output': ['xi_sl', 'gamma_sl'],
    'N_sl': [12, 12],
    'dot_gamma_0_sl': [0.001, 0.001],
    'n_sl': [20, 20],
    'a_sl': [2.25, 2.25],
    'xi_

In [15]:
gridfile.save("Steel_ebsd")
CM.save("Steel_ebsd.yaml")

In [11]:
vti_path = "Steel_ebsd.vti"
material_path = "Steel_ebsd.yaml"
v = damask.VTK.load(vti_path)
material_ID = v.get(label='material').flatten()

ma = damask.ConfigMaterial.load(material_path)

phases = list(ma['phase'].keys())
info = []

for m in ma['material']:
    c = m['constituents'][0]
    phase = c['phase']
    info.append({'phase':   phase,
                 'phaseID': phases.index(phase),
                 'lattice': ma['phase'][phase]['lattice'],
                 'O':       c['O'],
                })

l = np.array([0,0,1])                            # lab frame direction for IPF

IPF = np.ones((len(material_ID),3),np.uint8)
for i,data in enumerate(info):
    IPF[np.where(material_ID==i)] = \
    np.uint8(damask.Orientation(data['O'],lattice=data['lattice']).IPF_color(l)*255)

v = v.set(f'IPF_{l}',IPF)

p   = np.array([d['phase'] for d in info])
pid = np.array([d['phaseID'] for d in info])
v = v.set(label='phase',data=p[material_ID],info='phase name')
v = v.set(label='phaseID',data=pid[material_ID],info='phase ID')

v.save(f'IPF_{l}_Steel_ebsd.vti')

In [27]:
result_file = "Cu_ebsd_tensionX_Cu_ebsd_effectiveNumerics.hdf5"

res = damask.Result(result_file)

In [28]:
res

Created by DAMASK_grid v3.0.2
        on 2025-09-20 13:55:15+0800
 executing "DAMASK_grid --load tensionX.yaml --geom Cu_ebsd.vti --material Cu_ebsd.yaml --numerics effectiveNumerics.yaml"

increment_0 (0.0 s)
  phase
    Cu
      mechanical
        F / 1: deformation gradient
        F_e / 1: elastic deformation gradient
        L_p / 1/s: plastic velocity gradient
        O / 1: crystal orientation as quaternion q_0 (q_1 q_2 q_3)
        P / Pa: first Piola-Kirchhoff stress
        gamma_sl / 1: plastic shear
        xi_sl / Pa: resistance against plastic slip
  homogenization
    SX
      mechanical

increment_50
  ...

increment_100 (5.0 s)
  phase
    Cu
      mechanical
        F / 1: deformation gradient
        F_e / 1: elastic deformation gradient
        L_p / 1/s: plastic velocity gradient
        O / 1: crystal orientation as quaternion q_0 (q_1 q_2 q_3)
        P / Pa: first Piola-Kirchhoff stress
        gamma_sl / 1: plastic shear
        xi_sl / Pa: resistance against p

In [30]:
try:
    res.add_stress_Cauchy()
except ValueError:
    pass
try:
    res.add_equivalent_Mises('sigma')
except ValueError:
    pass
try:
    res.add_stress_second_Piola_Kirchhoff()
except ValueError:
    pass
try:
    res.add_strain()
except ValueError:
    pass
try:
    res.add_equivalent_Mises('epsilon_V^0.0(F)')
except ValueError:
    pass

In [33]:
res.add_IPF_color(l = [0,0,1], q = 'O')

In [34]:
res.export_VTK()